<a href="https://colab.research.google.com/github/elishaROBINSON/toys/blob/master/auto_translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:
%pip install pytube pydub numpy wavio tqdm wave
%pip install -q git+https://github.com/facebookresearch/seamless_communication

import argparse
import logging
import torch
import torchaudio
import wave
from seamless_communication.models.inference import Translator
from fairseq2.memory import MemoryBlock
from pydub import AudioSegment
import io
from tqdm import tqdm
from fairseq2.memory import MemoryBlock
import torchaudio
import torchaudio.transforms as T
from torchaudio.io import StreamReader
from pytube import YouTube
import os
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s %(levelname)s -- %(name)s: %(message)s",
)
url = "https://www.youtube.com/watch?v=lQDcmJZggqM"

logger = logging.getLogger(__name__)

def Download(link):
    youtubeObject = YouTube(link)
    youtubeObject = youtubeObject.streams.get_highest_resolution()
    try:
        youtubeObject.download(filename="input.mp4")
    except:
        print("An error has occurred")
    print("Download is completed successfully")

Download()

os.system('ffmpeg -i input.mp4 -vn -acodec pcm_s16le -ar 44100 -ac 2 test_file.wav')
waveform, sample_rate = torchaudio.load("test_file.wav")

def translate(wav_data):
    if torch.cuda.is_available():
        device = torch.device("cuda:0")
        dtype = torch.float16
        logger.info(f"Running inference on the GPU in {dtype}.")
    else:
        device = torch.device("cpu")
        dtype = torch.float32
        logger.info(f"Running inference on the CPU in {dtype}.")

    translator = Translator('seamlessM4T_large', 'vocoder_36langs', device, dtype)
    translated_text, wav, sr = translator.predict(
        wav_data,
        "s2st",
        "eng",
        src_lang=None,
        spkr=2,
        sample_rate=sample_rate,
        ngram_filtering=True,
    )
    return wav, sr

chunk_duration = 25

# Convert chunk duration to samples
chunk_samples = int(chunk_duration * sample_rate)
store = torch.empty(0,1, dtype=torch.float32, device='cuda:0')


for chunk in tqdm(range(0,waveform.size(1),chunk_samples)):
  val = waveform[:,chunk:chunk+chunk_samples].to('cuda:0')
  val = val.t()
  wav, sr = translate(val)
  store = torch.cat((store,wav[0].t()),dim=0)

torchaudio.save(
          f"translated_out.wav",
          store.t().to(torch.float32).cpu(),
          sample_rate=int(sr/3),
      )


  0%|          | 0/70 [00:00<?, ?it/s]Using the cached checkpoint of the model 'seamlessM4T_large'. Set `force=True` to download again.
Using the cached tokenizer of the model 'seamlessM4T_large'. Set `force=True` to download again.
Using the cached checkpoint of the model 'vocoder_36langs'. Set `force=True` to download again.
  1%|▏         | 1/70 [00:11<13:27, 11.70s/it]Using the cached checkpoint of the model 'seamlessM4T_large'. Set `force=True` to download again.
Using the cached tokenizer of the model 'seamlessM4T_large'. Set `force=True` to download again.
Using the cached checkpoint of the model 'vocoder_36langs'. Set `force=True` to download again.
  3%|▎         | 2/70 [00:21<12:00, 10.59s/it]Using the cached checkpoint of the model 'seamlessM4T_large'. Set `force=True` to download again.
Using the cached tokenizer of the model 'seamlessM4T_large'. Set `force=True` to download again.
Using the cached checkpoint of the model 'vocoder_36langs'. Set `force=True` to download agai

TypeError: ignored

In [109]:
torchaudio.save(
          f"translated_out.wav",
          store.t().to(torch.float32).cpu(),
          sample_rate=int(sr/3),
      )

In [101]:
sr


44100